## Inverse Cooking: Recipe Generation from Food Images

In [3]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from args import get_parser
import pickle
from model import get_model
from torchvision import transforms
from utils.output_utils import prepare_output
from PIL import Image
import time
from shutil import copyfile
import csv

In [4]:
vocab_file = '/home/ct2020dl5787/VireoFood172/vocab172_mapping.csv'
chinese_vocab = []
mapped_vocab = []
with open(vocab_file) as file:
    csv_reader = csv.reader(file, delimiter=',')
    for row in csv_reader:
        #check duplicate
        chinese_vocab.append(row[0])
        mapped_vocab.append(row[1])
        #print("originally: ", row[0], " mapped to ", row[1])

In [5]:
chinese_vocab

['Minced green onion',
 'chiffonade of green onion',
 'Seared green onion',
 'White onion',
 'Brunoise diced ham',
 'Julienned ham',
 'Ham',
 'Sliced ham',
 'Brunoise diced carrot',
 'Sliced carrot',
 'Batonnet carrot',
 'Julienned carrot',
 'Hob blocks of carrot',
 'Black sesame',
 'White sesame',
 'Crushed pepper',
 'Shredded pepper',
 'Pepper slices',
 'Seared pepper',
 'Pepper',
 'Whole green pepper',
 'Chinese Parsleycoriander',
 'Bean sprouts',
 'Bacon',
 'Tomato sclices',
 'Cherry tomato slices',
 'Cherry tomato',
 'Green vegetables',
 'chiffonade of green vegetables',
 'Corn kernels',
 'Corn blocks',
 'Bitter gourd slices',
 'Broccoli',
 'Cauliflower',
 'Toast',
 'Bread',
 'Chinese mahogany',
 'Crab sticks',
 'Bamboo shoots tips',
 'Chive pieces',
 'Chopped chives',
 'Chives',
 'Shredded onion',
 'Onion slices',
 'Brunoise diced onion',
 'Pumpkin blocks',
 'Groundnut kernels',
 'Crushed groundnut kernels',
 'Lemon',
 'Shelled fresh shrimps',
 'Dried sea shrimp',
 'Fresh Shrimp'

In [6]:
true_labels = dict()
ing_labels_file = open('/home/ct2020dl5787/VireoFood172/SplitAndIngreLabel/IngreLabel.txt', 'r')
contents = ing_labels_file.readlines()
for line in contents:
    filename = line.split()[0]
    labels = line.split()[1:]
    true_labels[filename] = labels
ing_labels_file.close()

In [7]:
img_names = []
img_ingre = []
img_chinese_ingre = []

image_dir = '/home/ct2020dl5787/VireoFood172/sample_pictures1720'
for f in os.listdir(image_dir):
    name = "/"+f
    name = name.replace("_","/",1)
    if name not in true_labels:
        print(name)
    else :
        binary_label = true_labels[name]
        mapped_labels = []
        mapped_chinese_labels = []
        for index, label in enumerate(binary_label):
            if(label == "1") :
                mapped_labels.append(mapped_vocab[index])
                mapped_chinese_labels.append(chinese_vocab[index])
        
        img_ingre.append(mapped_labels)
        img_chinese_ingre.append(mapped_chinese_labels)
        img_names.append(f.replace("/", "_"))

/layer1.json
/layer2.json


In [18]:
img_dict = {'img_names': img_names, 'true_ingredients': img_ingre, 'ingredients_Chinese' : img_chinese_ingre}  
true_label_df = pd.DataFrame(img_dict) 

In [19]:
true_label_df

,img_names,true_ingredients,ingredients_Chinese
0,86_10102721546691.jpg,[crab],[Crab]
1,140_10_14.jpg,"[diced_green_chilis, chili, garlic, squid]","[Crushed pepper, Crushed hot and dry chili, Cr..."
2,100_10_2.jpg,"[ham, egg, mushroom]","[Sliced ham, Egg cake, Brunoise diced lentinus..."
3,64_0fd9b207887e2199159d73db9cc5a803.jpg,"[scallion, pepper, tofu]","[Seared green onion, Shredded pepper, Tofu chu..."
4,160_10_13.jpg,"[black_sesame_seed, diced_green_chilis, Parsle...","[Black sesame, Crushed pepper, Chinese Parsley..."
...,...,...,...
1715,109_10_24.jpg,"[black_sesame_seed, Parsley, peanuts, chili, d...","[Black sesame, Chinese Parsleycoriander, Groun..."
1716,84_10_2.jpg,"[greens, tofu]","[Green vegetables, Stinky tofu]"
1717,111_10_21.jpg,"[scallion, carrot, Parsley, chili, dried_black...","[Minced green onion, Hob blocks of carrot, Chi..."
1718,135_10_10.jpg,"[greens, bean_threads, bean_curd]","[Green vegetables, Rice noodle, Fried yuba skin]"


In [20]:
#Build JSON
import json

In [29]:
#First layer1.json
layer1_file_path = image_dir + "/layer1.json"
layer1_list = []
img_len = 1720
for i in range(img_len):
    img_name = true_label_df['img_names'][i]
    jpg_index = img_name.find(".jpg")
    img_id = img_name[:jpg_index]
    ingredients = true_label_df['true_ingredients'][i]
    #split the dataset so train:val:test is 6:1:1
    if (i%8 == 1):
        split = "val"
    elif(i%8 == 2):
        split = "test"
    else:
        split = "train"
    
    ingr_list = []
    for ingr in ingredients:
        ingr_dict = {"text": ingr}
        ingr_list.append(ingr_dict)
    
    img_dict = {"id": img_id, 
                "title": img_name,
                "instructions": [],
                "ingredients": ingr_list,
                "partition":split,
                "url":""}
    layer1_list.append(img_dict)
    
with open(layer1_file_path, 'w') as json_file:
    json.dump(layer1_list, json_file)

In [30]:
layer1_list[5]

{'id': '75_1008209.1',
 'title': '75_1008209.1.jpg',
 'instructions': [],
 'ingredients': [{'text': 'garlic'}, {'text': 'spinach'}],
 'partition': 'train',
 'url': ''}

In [31]:
#Then layer2.json
layer2_file_path = image_dir + "/layer2.json"
layer2_list = []
img_len = 1720
for i in range(img_len):
    img_name = true_label_df['img_names'][i]
    jpg_index = img_name.find(".jpg")
    img_id = img_name[:jpg_index]
    
    img_dict = {"id": img_id, 
                "images": [{"id": img_id,"url":""}]}
    layer2_list.append(img_dict)
    
with open(layer2_file_path, 'w') as json_file:
    json.dump(layer2_list, json_file)